In [2]:
import urllib.request
import pandas as pd

In [3]:
id_uni = 190

In [4]:
db_players = f'https://s{id_uni}-fr.ogame.gameforge.com/api/players.xml'  # une fois par jour
db_ladder = f'https://s{id_uni}-fr.ogame.gameforge.com/api/highscore.xml?category=1&type=1' # par heure
# category : 1 = Player, 2 = Alliance
type = {0 : 'Total',
        1 : 'Eco',
        2 : 'Research',
        3 : 'Militaire',
        4 : 'Militaire lost',
        5 : 'Militaire construit',
        6 : 'Militaire Détruit',
        7 : 'Honor',
        8 : 'FdV',
        9 : 'Eco FdV',
        10 : 'Techno FdV',
        11 : 'Exploration FdV'}

# On récupère la data

In [5]:
urllib.request.urlretrieve(db_players, "xml/players.xml")

('xml/players.xml', <http.client.HTTPMessage at 0x1dd634e8940>)

In [6]:
for key, value in type.items():
    db = f'https://s{id_uni}-fr.ogame.gameforge.com/api/highscore.xml?category=1&type={key}'
    urllib.request.urlretrieve(db, f"xml/ladder_{value}.xml")

# Chargement

In [7]:
df_players = pd.read_xml('xml/players.xml')

# Modif

In [8]:
# Osef de l'alliance
df_players.drop('alliance', axis=1, inplace=True)

In [9]:
mapping = {'a' : 'admin',
           'v' : 'mv',
           'i' : 'inactif',
           None: 'actif'}
df_players['status'] = df_players['status'].replace(mapping)
df_players

,id,name,status
0,1,Legor,admin
1,100000,GameAdmin,admin
2,100003,comagf,admin
3,100004,OGameTeam,admin
4,100006,Maki,actif
...,...,...,...
4574,107272,Fallrock,actif
4575,107273,Chancellor Eris,actif
4576,107274,Procurator Moon,actif
4577,107275,Technocrat Scout,actif


# On fusionne

In [10]:
for value in type.values():
    df = pd.read_xml(f'xml/ladder_{value}.xml')
    if len(df.columns) == 3:
        df.columns = [f'position_{value}', 'id', f'points_{value}']
    else:
        df.columns = [f'position_{value}', 'id', f'points_{value}', "ships"]
    df_players = pd.merge(df_players, df, on='id')

In [11]:
df_players

,id,name,status,position_Total,points_Total,position_Eco,points_Eco,position_Research,points_Research,position_Militaire,...,position_Honor,points_Honor,position_FdV,points_FdV,position_Eco FdV,points_Eco FdV,position_Techno FdV,points_Techno FdV,position_Exploration FdV,points_Exploration FdV
0,100006,Maki,actif,53,6043628,120,1779891,95,443654,24,...,4578,-227594,94,321332,91,256445,113,64861,135,26
1,100008,JackDetta,actif,946,128321,1012,65487,827,22104,774,...,322,3765,821,7701,724,7648,1234,50,880,2
2,100009,dylou,inactif,1504,31060,1387,22619,1429,5850,2451,...,688,722,1075,2589,998,2589,1398,0,1118,0
3,100010,Belle Alice,mv,1073,95211,1298,30538,1390,6505,711,...,1538,1,1590,334,1547,334,1399,0,1119,0
4,100011,Dragibus,actif,80,4474456,145,1464160,188,226943,38,...,4538,-44854,163,177099,138,160269,318,16808,183,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,107272,Fallrock,actif,2397,1002,2392,630,2410,171,2070,...,4185,0,2499,1,2498,1,4516,0,4516,0
4516,107273,Chancellor Eris,actif,4059,0,4073,0,4529,0,4523,...,4186,0,3670,0,3669,0,4517,0,4517,0
4517,107274,Procurator Moon,actif,3114,16,3105,14,2973,1,4524,...,4187,0,3272,0,3271,0,4518,0,4518,0
4518,107275,Technocrat Scout,actif,4558,0,4558,0,4530,0,4525,...,4188,0,4551,0,4551,0,4519,0,4519,0


# On cherche les meilleurs ina

In [12]:
pd.set_option('display.max_rows', None)
df_ina = df_players[df_players['status'] == 'inactif']
df_ina.sort_values('points_Total', inplace=True, ascending=False)
df_ina.to_excel('inactif/ina.xlsx', index=False)
df_ina.head(30)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_9884\191281827.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ina.sort_values('points_Total', inplace=True, ascending=False)


,id,name,status,position_Total,points_Total,position_Eco,points_Eco,position_Research,points_Research,position_Militaire,...,position_Honor,points_Honor,position_FdV,points_FdV,position_Eco FdV,points_Eco FdV,position_Techno FdV,points_Techno FdV,position_Exploration FdV,points_Exploration FdV
2553,104098,zyldrac,inactif,254,1409344,249,808302,197,214595,299,...,44,28191,73,386445,78,284270,67,102175,2873,0
746,101070,Rheve89,inactif,292,1074146,265,714693,226,183680,694,...,88,18215,172,167012,183,116236,139,50747,118,29
968,101412,Wyatt Earp,inactif,302,1033344,288,639725,136,332734,1415,...,69,21009,338,60884,304,52699,453,8166,240,18
1652,102644,Superfly,inactif,306,1003312,248,811184,289,129901,1301,...,403,2483,352,55706,334,43013,359,12687,689,5
176,100242,Black Ghost,inactif,315,951083,379,397270,257,151177,1329,...,86,18266,71,394709,52,386165,441,8518,137,26
1418,102173,Doug Wilson,inactif,367,756209,316,540108,238,176089,1909,...,242,6285,427,39411,375,34466,570,4937,515,8
1220,101819,Mini-Katana,inactif,370,748672,327,499616,200,208348,1543,...,280,4895,441,36667,555,15433,269,21227,618,6
390,100556,Nemaussan,inactif,409,618652,353,446585,271,142233,630,...,188,8555,730,11193,764,6602,587,4583,500,8
43,100060,Tolbiac,inactif,443,549391,435,332141,253,158354,1065,...,110,14849,388,46707,497,19352,224,27343,386,11
3478,105974,DrChaos,inactif,519,416894,533,245892,417,73636,538,...,503,1637,254,93726,267,62967,206,30743,304,15


# Carac uni

In [28]:
db_liste_uni = 'https://s190-fr.ogame.gameforge.com/api/universes.xml'
urllib.request.urlretrieve(db_liste_uni, f"./xml/liste_univers.xml")

df_liste_uni = pd.read_xml('./xml/liste_univers.xml')


for num_uni in df_liste_uni['id'].values:

    db_uni = f'https://s{num_uni}-fr.ogame.gameforge.com/api/serverData.xml'
    urllib.request.urlretrieve(db_uni, f"./xml/uni/{num_uni}.xml")


    

  

In [76]:
# pd.read_xml('./xml/uni/123.xml')

import xml.etree.ElementTree as et 


In [91]:
name_uni = []
speed_uni = []
speed_flotte_peace = []
speed_flotte_war = []
speed_flotte_holding = []
cdr_flotte = []
cdr_def = []
top1 = []

for num_uni in df_liste_uni['id'].values:
    my_tree = et.parse(f'./xml/uni/{num_uni}.xml')
    my_root = my_tree.getroot()
    name_uni.append(my_root[0].text)
    speed_uni.append(my_root[7].text)
    speed_flotte_peace.append(my_root[8].text)
    speed_flotte_war.append(my_root[9].text)
    speed_flotte_holding.append(my_root[10].text)
    cdr_flotte.append(my_root[16].text)
    cdr_def.append(my_root[17].text)
    top1.append(my_root[21].text)
    
    
df_univers = pd.DataFrame([name_uni, speed_uni, speed_flotte_peace, speed_flotte_war, speed_flotte_holding, cdr_flotte, cdr_def, top1],
                          index=['Name', 'Vitesse eco', 'Vitesse allie', 'Vitesse attaque', 'Vitesse statio', 'CDR Flotte', 'CDR Def', 'Top1']).transpose()   


    

,Name,Vitesse eco,Vitesse allie,Vitesse attaque,Vitesse statio,CDR Flotte,CDR Def,Top1
0,Wasat,7,2,2,2,0.5,0,4476944366.35
1,Eridanus,7,5,5,5,0.5,0,5728666537.9911
2,Libra,1,1,1,1,0.3,0,921759931.311
3,Nusakan,7,1,1,1,0.7,0,5236162232.293
4,Quaoar,7,1,1,1,0.7,0,3352254922.006
5,Kalyke,4,2,2,2,0.5,0,516176613.522
6,Leda,1,1,1,1,0.5,0,177114960.21
7,Octans,7,3,3,3,0.6,0,3724832561.71
8,Tucana,10,3,3,3,0.4,0,67965133032.474
9,Volans,7,4,4,4,0.8,0,1746935721


In [94]:
df_univers['Top1'] = df_univers['Top1'].astype('float')

In [89]:
df_univers[df_univers['Name'] == 'Wasat']['Vitesse eco'].values[0]

'7'